<a href="https://colab.research.google.com/github/MehrdadDastouri/gan_mnist/blob/main/gan_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Hyperparameters
latent_size = 64
hidden_size = 256
image_size = 28 * 28
batch_size = 128
learning_rate = 0.0002
epochs = 100

# Data preparation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

# Define the Generator
class Generator(nn.Module):
    def __init__(self, latent_size, hidden_size, image_size):
        super(Generator, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(latent_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, image_size),
            nn.Tanh()  # Output normalized to [-1, 1]
        )

    def forward(self, z):
        return self.net(z)

# Define the Discriminator
class Discriminator(nn.Module):
    def __init__(self, image_size, hidden_size):
        super(Discriminator, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(image_size, hidden_size),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_size, hidden_size),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_size, 1),
            nn.Sigmoid()  # Output probability
        )

    def forward(self, x):
        return self.net(x)

# Initialize the Generator and Discriminator
generator = Generator(latent_size, hidden_size, image_size).to(device)
discriminator = Discriminator(image_size, hidden_size).to(device)

# Loss function and optimizers
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer_g = optim.Adam(generator.parameters(), lr=learning_rate)
optimizer_d = optim.Adam(discriminator.parameters(), lr=learning_rate)

# Function to denormalize images
def denormalize(img):
    return (img + 1) / 2

# Training the GAN
fixed_noise = torch.randn(16, latent_size).to(device)  # Fixed noise for visualization
d_losses, g_losses = [], []

for epoch in range(epochs):
    d_loss_epoch, g_loss_epoch = 0, 0

    for real_images, _ in train_loader:
        real_images = real_images.view(-1, image_size).to(device)

        # Create labels
        real_labels = torch.ones(real_images.size(0), 1).to(device)
        fake_labels = torch.zeros(real_images.size(0), 1).to(device)

        # Train the Discriminator
        outputs = discriminator(real_images)
        d_loss_real = criterion(outputs, real_labels)

        noise = torch.randn(real_images.size(0), latent_size).to(device)
        fake_images = generator(noise)
        outputs = discriminator(fake_images.detach())  # Detach to avoid training the generator
        d_loss_fake = criterion(outputs, fake_labels)

        d_loss = d_loss_real + d_loss_fake
        optimizer_d.zero_grad()
        d_loss.backward()
        optimizer_d.step()

        # Train the Generator
        noise = torch.randn(real_images.size(0), latent_size).to(device)
        fake_images = generator(noise)
        outputs = discriminator(fake_images)
        g_loss = criterion(outputs, real_labels)  # Try to trick the discriminator

        optimizer_g.zero_grad()
        g_loss.backward()
        optimizer_g.step()

        d_loss_epoch += d_loss.item()
        g_loss_epoch += g_loss.item()

    d_losses.append(d_loss_epoch / len(train_loader))
    g_losses.append(g_loss_epoch / len(train_loader))

    print(f"Epoch [{epoch+1}/{epochs}], D Loss: {d_losses[-1]:.4f}, G Loss: {g_losses[-1]:.4f}")

    # Visualize generated images
    if (epoch + 1) % 10 == 0:
        with torch.no_grad():
            fake_images = generator(fixed_noise).view(-1, 1, 28, 28)
            fake_images = denormalize(fake_images).cpu()

        plt.figure(figsize=(8, 8))
        for i in range(16):
            plt.subplot(4, 4, i + 1)
            plt.imshow(fake_images[i][0], cmap="gray")
            plt.axis("off")
        plt.suptitle(f"Generated Images at Epoch {epoch+1}")
        plt.show()

# Plot losses
plt.figure(figsize=(10, 5))
plt.plot(d_losses, label="Discriminator Loss")
plt.plot(g_losses, label="Generator Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training Losses")
plt.legend()
plt.show()

Using device: cuda
Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 12.7MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 345kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 3.18MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 3.18MB/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch [1/100], D Loss: 0.2517, G Loss: 3.7208
Epoch [2/100], D Loss: 0.0788, G Loss: 5.0290
Epoch [3/100], D Loss: 0.2252, G Loss: 4.6909
Epoch [4/100], D Loss: 0.2924, G Loss: 4.1068
Epoch [5/100], D Loss: 0.2535, G Loss: 4.1288
Epoch [6/100], D Loss: 0.1666, G Loss: 4.7572
Epoch [7/100], D Loss: 0.2411, G Loss: 4.8936
Epoch [8/100], D Loss: 0.2362, G Loss: 4.6106
